In [6]:
# !pip install google-cloud-firestore
# !pip install --quiet imageio-ffmpeg
# !pip install google-cloud-storage
# !pip install google-cloud-speech
!pip install pydub ffmpeg-python



   ---------------------------------------- 0/3 [pydub]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- -------------------------- 1/3 [future]
   ------------- --------------

In [2]:
# import os
# from google.cloud import firestore


# db = firestore.Client()

# # # # Create / update a document
# # doc_ref = db.collection("users").document("alice")
# # doc_ref.set({"name": "Alice", "age": 30, "city": "London"})

# # Read a document
# doc = db.collection("users").document("alice").get()
# if doc.exists:
#     print("Document data:", doc.to_dict())

# # Query a collection
# users_ref = db.collection("users")
# query = users_ref.where("age", ">", 20).stream()
# for user in query:
#     print(f"{user.id} => {user.to_dict()}")

# # # Delete a document
# db.collection("users").document("alice").delete()


In [ ]:
from google.cloud import firestore
db = firestore.Client()

users_ref = db.collection("users_data")
# Stream all documents
docs = users_ref.stream()

for doc in docs:
    print(f"{doc.id} => {doc.to_dict()}")


In [4]:
import os
import re
import mimetypes
from pathlib import Path
from typing import Optional
from datetime import datetime, timezone
from google.cloud import storage

import subprocess
from pathlib import Path

import requests

In [23]:


# If you haven't set this in your environment, do it here:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials.json"

ALLOWED_PREFIXES = ("audio/", "video/")

_slugify_re = re.compile(r"[^a-zA-Z0-9._-]+")

def _slugify(s: str) -> str:
    """
    Convert arbitrary text to a safe path segment.
    Keeps letters, digits, dot, underscore, dash. Replaces others with '-'.
    """
    s = s.strip()
    s = _slugify_re.sub("-", s)
    s = re.sub(r"-{2,}", "-", s).strip("-")
    return s or "untitled"

def upload_media_to_gcs(
    bucket_name: str,
    local_path: str,
    username: str,
    meeting_title: str,
    make_public: bool = False,
    return_signed_url: bool = False,
    signed_url_ttl_seconds: int = 3600,
    chunk_size_mb: int = 8,
) -> dict:
    """
    Upload an audio/video file to GCS.

    Destination path pattern:
        username/<meeting_title>_<YYYYMMDD_HHMMSS>.mp4

    Args:
        bucket_name: GCS bucket name (no gs://).
        local_path: Path to the local audio/video file.
        username: User identifier (becomes the top-level folder).
        meeting_title: Human-readable title (used in the filename).
        make_public: If True, makes the object publicly readable.
        return_signed_url: If True, returns a time-limited signed URL.
        signed_url_ttl_seconds: Expiry for the signed URL (seconds).
        chunk_size_mb: Resumable upload chunk size (MB).

    Returns:
        dict with 'bucket', 'object', 'size', 'content_type', and urls.
    """
    p = Path(local_path)
    if not p.is_file():
        raise FileNotFoundError(f"File not found: {local_path}")

    # Guess MIME type and enforce audio/video input
    ctype, _ = mimetypes.guess_type(p.name)
    if not ctype or not ctype.startswith(ALLOWED_PREFIXES):
        raise ValueError(f"File must be audio/video. Detected content-type: {ctype!r}")

    # Compose destination path: username/meeting_title_YYYYMMDD_HHMMSS.mp4
    user_seg = _slugify(username)
    title_seg = _slugify(meeting_title)
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    blob_name = f"{user_seg}/{title_seg}_{ts}.mp4"

    # GCS client and blob
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    # Resumable upload chunk size
    blob.chunk_size = chunk_size_mb * 1024 * 1024

    # Upload (we keep original content_type even if the object name ends with .mp4)
    blob.upload_from_filename(filename=str(p), content_type=ctype)

    # Optional ACL
    public_url = None
    if make_public:
        blob.make_public()
        public_url = blob.public_url

    # Optional signed URL
    signed_url = None
    if return_signed_url:
        signed_url = blob.generate_signed_url(
            expiration=signed_url_ttl_seconds, method="GET"
        )

    return {
        "bucket": bucket_name,
        "object": blob_name,
        "size": p.stat().st_size,
        "content_type": ctype,
        "gs_uri": f"gs://{bucket_name}/{blob_name}",
        "public_url": public_url,
        "signed_url": signed_url,
    }



def download_file(public_url: str, local_filename: str):
    """
    Download a file from a public URL and save it locally.
    """
    resp = requests.get(public_url, stream=True)
    resp.raise_for_status()  # raise error if download failed

    with open(local_filename, "wb") as f:
        for chunk in resp.iter_content(chunk_size=8192):
            if chunk:  # skip keep-alive chunks
                f.write(chunk)

    print(f"Downloaded to {local_filename}")
    
    
# def _get_ffmpeg_path() -> str:
#     """Return a usable ffmpeg executable path."""
#     try:
#         subprocess.run(["ffmpeg", "-version"], check=True, capture_output=True)
#         return "ffmpeg"
#     except Exception:
#         try:
#             import imageio_ffmpeg
#             return imageio_ffmpeg.get_ffmpeg_exe()
#         except Exception as e:
#             raise RuntimeError(
#                 "ffmpeg not found. Install with `apt-get install ffmpeg` or `pip install imageio-ffmpeg`."
#             ) from e

# def split_mp4(input_file: str):
#     """
#     Split an MP4 into audio-only and video-only files.
#     Output filenames: <basename>_audio.mp4 and <basename>_video.mp4

#     Returns:
#         (audio_file, video_file)
#     """
#     in_path = Path(input_file)
#     if not in_path.is_file():
#         raise FileNotFoundError(f"File not found: {input_file}")

#     base = in_path.with_suffix("")  # strip extension
#     audio_out = f"{base}_audio.mp4"
#     video_out = f"{base}_video.mp4"

#     ffmpeg = _get_ffmpeg_path()

#     # Extract audio (re-mux into mp4 container)
#     subprocess.run(
#         [ffmpeg, "-y", "-i", str(in_path), "-vn", "-acodec", "copy", audio_out],
#         check=True
#     )

#     # Extract video (mute audio)
#     subprocess.run(
#         [ffmpeg, "-y", "-i", str(in_path), "-an", "-vcodec", "copy", video_out],
#         check=True
#     )

#     return audio_out, video_out

import subprocess
from pathlib import Path

def _get_ffmpeg_path():
    """Helper to get ffmpeg binary, customize if needed."""
    return "ffmpeg"  # Assumes ffmpeg is in PATH

def split_mp4(input_file: str):
    """
    Split an MP4 into a WAV audio file and a video-only MP4 file.
    Output filenames: <basename>_audio.wav and <basename>_video.mp4

    Returns:
        (audio_file, video_file)
    """
    in_path = Path(input_file)
    if not in_path.is_file():
        raise FileNotFoundError(f"File not found: {input_file}")

    base = in_path.with_suffix("")  # strip extension
    audio_out = f"{base}_audio.wav"
    video_out = f"{base}_video.mp4"

    ffmpeg = _get_ffmpeg_path()

    # Extract audio to WAV (recommended for Speech-to-Text)
    subprocess.run(
        [
            ffmpeg, "-y", "-i", str(in_path),
            "-vn",               # no video
            "-acodec", "pcm_s16le",  # WAV format
            "-ar", "16000",      # sample rate: 16 kHz
            "-ac", "1",          # mono channel
            audio_out
        ],
        check=True
    )

    # Extract video (mute audio)
    subprocess.run(
        [
            ffmpeg, "-y", "-i", str(in_path),
            "-an",              # no audio
            "-vcodec", "copy",
            video_out
        ],
        check=True
    )

    return audio_out, video_out


In [27]:
result = upload_media_to_gcs(
    bucket_name="ai_meeting_recording",
    local_path="sample.mp4",
    username="ali",
    meeting_title="weekly sync",
    make_public=True,
    return_signed_url=True,
    signed_url_ttl_seconds=3600,
)
print(result)

public_url = result["public_url"]

download_file(public_url, "temp.mp4")

audio_file, video_file = split_mp4("temp.mp4")

{'bucket': 'ai_meeting_recording', 'object': 'ali/weekly-sync_20250824_093341.mp4', 'size': 2097152, 'content_type': 'video/mp4', 'gs_uri': 'gs://ai_meeting_recording/ali/weekly-sync_20250824_093341.mp4', 'public_url': 'https://storage.googleapis.com/ai_meeting_recording/ali/weekly-sync_20250824_093341.mp4', 'signed_url': 'https://storage.googleapis.com/ai_meeting_recording/ali/weekly-sync_20250824_093341.mp4?Expires=3600&GoogleAccessId=474435376108-compute%40developer.gserviceaccount.com&Signature=iL6jhfd9TRxsLVHznHWbJlGI9f%2Fj9Soj7dWgIgult0MuKVN84E%2BaEvYjK4pjF0yC0KcrQFfC%2FAxDi895JnyGSPB7wMeyx7o20JoqyuIlEk6%2BhuoRKNn6fAfFr7pcimfJnopd4j9pQqqZ4zd3ozCgchXpMF9Ky%2BYjt8xET61nFcZnKnxKVBbP1LPT1G3igLMqrn5CkD%2FxQo3uLE%2B5LGXh5oQFU5AVZPsOtQQNrU6wztxpVdbgi74kHwJWhRi722G09YISncasj1X%2BhhV1Yi0FwM5P80xzsryldJs%2Fc0aqlmaMWHtDsmOZic1hkhfsKiaFrbcHJOzT9tvd0DEGYjKvcw%3D%3D'}
Downloaded to temp.mp4


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
# 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

MODEL_ID="chirp_2"
PROJECT_ID="aimeetingassistant-470006"

In [ ]:
from google.cloud import speech

client = speech.SpeechClient.from_service_account_json('credentials.json')

gcs_uri = 'gs://your-bucket-name/path/sample.wav'

audio = speech.RecognitionAudio(uri=gcs_uri)
config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code='en-US',
)

operation = client.long_running_recognize(config=config, audio=audio)

print("Waiting for operation to complete...")
response = operation.result(timeout=600)

for result in response.results:
    print(f"Transcript: {result.alternatives[0].transcript}")
    print(f"Confidence: {result.alternatives[0].confidence}")


InvalidArgument: 400 Sync input too long. For audio longer than 1 min use LongRunningRecognize with a 'uri' parameter.

In [28]:
!pip install moviepy

   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ------ --------------------------------- 1.0/7.0 MB 4.9 MB/s eta 0:00:02
   ---------- ----------------------------- 1.8/7.0 MB 6.0 MB/s eta 0:00:01
   ------------------ --------------------- 3.1/7.0 MB 5.7 MB/s eta 0:00:01
   ------------------ --------------------- 3.1/7.0 MB 5.7 MB/s eta 0:00:01
   ------------------------- -------------- 4.5/7.0 MB 4.4 MB/s eta 0:00:01
   ------------------------------- -------- 5.5/7.0 MB 4.3 MB/s eta 0:00:01
   --------------------------------- ------ 5.8/7.0 MB 3.8 MB/s eta 0:00:01
   ------------------------------------ --- 6.3/7.0 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 3.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.8 MB 8.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.8/12.8 MB 4.9 MB/s eta 0:00:03
   --------- ----------

In [6]:
import ffmpeg

def extract_audio_ffmpeg_python(input_file, output_file):
    (
        ffmpeg
        .input(input_file)
        .output(output_file, format='wav', acodec='pcm_s16le', ac=1, ar=16000)
        .run(overwrite_output=True)
    )

extract_audio_ffmpeg_python('sample.mp4', 'sample_audio.wav')


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [7]:
!pip install python-vlc


In [8]:
import vlc
import time
import os

def extract_audio_vlc(input_path, output_path):
    instance = vlc.Instance()
    player = instance.media_player_new()

    media = instance.media_new(input_path, f":sout=#transcode{{acodec=s16l,channels=1,samplerate=16000}}:std{{access=file,mux=wav,dst={output_path}}}", ":no-sout-all", ":sout-keep")
    player.set_media(media)

    player.play()

    # Wait for extraction to finish (simple sleep - adjust based on length)
    duration = 10  # seconds, adjust to length of your input
    time.sleep(duration)

    player.stop()

# Usage:
extract_audio_vlc('sample.mp4', 'sample_audio.wav')


FileNotFoundError: Could not find module 'c:\Users\d X b\Documents\aimeeting assistant\libvlc.dll' (or one of its dependencies). Try using the full path with constructor syntax.